In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from os import mkdir
from datetime import datetime
from os.path import exists, join
from itertools import chain

# import matplotlib.pyplot as plt
# import seaborn as sns
from utils import *

In [2]:
train_df = pd.read_pickle('../dataset/train.pkl')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1094826 entries, 0 to 1094825
Columns: 151 entries, orderid to hotel_roomid
dtypes: datetime64[ns](2), float16(117), float32(12), int16(3), int32(6), int8(11)
memory usage: 354.0 MB


In [2]:
test_df = pd.read_pickle('../dataset/test.pkl')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978298 entries, 0 to 978297
Columns: 150 entries, orderid to hotel_roomid
dtypes: datetime64[ns](2), float16(117), float32(12), int16(2), int32(6), int8(11)
memory usage: 314.4 MB


In [3]:
user_oreder_type = [x for x in train_df.columns if x.startswith('ordertype')]
user_orderbehavior = [x for x in train_df.columns if x.startswith('orderbehavior')]
user_lastord = [x for x in train_df.columns if x.endswith('lastord')]
user_feature_cols = ['uid'] + [x for x in train_df.columns if x.startswith('user')]
user_cols = list(chain(user_oreder_type, user_orderbehavior, user_lastord, user_feature_cols))

NameError: name 'train_df' is not defined

In [4]:
timespans = [str(x.date()) for x in pd.date_range('2017/6/5', '2017/6/11')]

In [5]:
for timespan in timespans:
    d = join('..', 'dataset', timespan[-2:])
    if not exists(d):
        mkdir(d)
test_dir = join('..', 'dataset', 'test')
if not exists(test_dir):
    mkdir(test_dir)

In [6]:
def save_id(tdf, t, timespan, is_test=False):
    f_dir = timespan if is_test else timespan[-2:]
    filename = join('..', 'dataset', f_dir, '{}.pkl'.format(t))
    print(datetime.now(), 'save id to', filename)
    sdf = tdf[[t]].drop_duplicates().reset_index(drop=True)
    sdf.to_pickle(filename)

In [7]:
def make_sample(train_df, timespan):
    tdf = train_df.set_index('orderdate').loc[timespan].reset_index()

    user_buy = tdf[['uid', 'orderid']].drop_duplicates().groupby('uid').count()

    buyone = user_buy.loc[user_buy.orderid==1]

    tdf = tdf.join(buyone, on='uid', rsuffix='_one')

    old_shape = tdf.shape[0]

    tdf = tdf.loc[tdf.orderid_one.notnull()]

    print(timespan, 'delete', old_shape - tdf.shape[0], 'sample')

    tdf.drop('orderid_one', axis=1, inplace=True)

    for t in ['uid', 'hotelid' , 'basicroomid', 'roomid', 'hotel_roomid']:
        save_id(tdf, t, timespan)

    base_save_path = join('..', 'dataset', timespan[-2:], 'base_feauture.pkl')

    tdf.to_pickle(base_save_path)

In [62]:
for timespan in timespans:
    make_sample(train_df, timespan)

2017-06-05 delete 2097 sample
2017-07-29 22:58:24.489618 save id to ../dataset/05/uid.pkl
2017-07-29 22:58:24.495044 save id to ../dataset/05/hotelid.pkl
2017-07-29 22:58:24.500069 save id to ../dataset/05/basicroomid.pkl
2017-07-29 22:58:24.506841 save id to ../dataset/05/roomid.pkl
2017-07-29 22:58:24.521819 save id to ../dataset/05/hotel_roomid.pkl
2017-06-06 delete 1542 sample
2017-07-29 22:58:24.987615 save id to ../dataset/06/uid.pkl
2017-07-29 22:58:24.998405 save id to ../dataset/06/hotelid.pkl
2017-07-29 22:58:25.003912 save id to ../dataset/06/basicroomid.pkl
2017-07-29 22:58:25.010868 save id to ../dataset/06/roomid.pkl
2017-07-29 22:58:25.026489 save id to ../dataset/06/hotel_roomid.pkl
2017-06-07 delete 1742 sample
2017-07-29 22:58:25.443349 save id to ../dataset/07/uid.pkl
2017-07-29 22:58:25.451644 save id to ../dataset/07/hotelid.pkl
2017-07-29 22:58:25.459161 save id to ../dataset/07/basicroomid.pkl
2017-07-29 22:58:25.467190 save id to ../dataset/07/roomid.pkl
2017-07

In [44]:
# save_id(tdf, 'basicroomid', timespan)

2017-07-29 22:30:14.683045 save to ../dataset/05/basicroomid.pkl


In [65]:
for t in ['uid', 'hotelid' , 'basicroomid', 'roomid', 'hotel_roomid']:
    save_id(test_df, t, 'test', is_test=True)

2017-07-29 23:00:20.753882 save id to ../dataset/test/uid.pkl
2017-07-29 23:00:20.781300 save id to ../dataset/test/hotelid.pkl
2017-07-29 23:00:20.807261 save id to ../dataset/test/basicroomid.pkl
2017-07-29 23:00:20.848147 save id to ../dataset/test/roomid.pkl
2017-07-29 23:00:20.995448 save id to ../dataset/test/hotel_roomid.pkl


In [12]:
test_file_path = join('..', 'dataset', 'test', 'base_feauture.pkl')

In [8]:
test_df['orderlabel'] = np.nan

In [10]:
test_df['orderlabel'] = test_df.orderlabel.astype(np.float16)

In [13]:
test_df.to_pickle(test_file_path)